In [1]:
import QuantLib as ql

In [2]:
s = ql.SimpleQuote(0.01)

In [3]:
s.value()

0.01

In [4]:
t = ql.SimpleQuote(0.02)

In [5]:
def f(x,y):
    return x+y

In [6]:
c = ql.CompositeQuote(ql.QuoteHandle(s),ql.QuoteHandle(t), f)

In [7]:
c = ql.CompositeQuote(ql.QuoteHandle(s),ql.QuoteHandle(t), lambda x,y:x+y)

In [8]:
c.value()

0.03

In [9]:
d = ql.DerivedQuote(ql.QuoteHandle(s),lambda x: 10*x)

In [10]:
d.value()

0.1

In [11]:
today = ql.Settings.instance().evaluationDate

In [12]:
dc = ql.Actual360()

In [13]:
spot = ql.SimpleQuote(100.0)

In [14]:
rTS = ql.FlatForward(today, 0.01, dc)

In [15]:
volTS = ql.BlackConstantVol(today, ql.NullCalendar(), 0.20, dc)

In [16]:
volHandle = ql.RelinkableBlackVolTermStructureHandle(volTS)

In [17]:
stochProcess = ql.BlackScholesProcess(ql.QuoteHandle(spot), ql.YieldTermStructureHandle(rTS), volHandle)

In [18]:
exerciseDate = today + ql.Period("6M")

In [19]:
payoff = ql.PlainVanillaPayoff(ql.Option.Call,100.0)

In [20]:
exercise = ql.EuropeanExercise(exerciseDate)

In [21]:
knockIn = ql.BarrierOption(ql.Barrier.DownIn, 90.0, 0.0, payoff, exercise)

In [22]:
knockOut = ql.BarrierOption(ql.Barrier.DownOut, 90.0, 0.0, payoff, exercise)

In [23]:
european = ql.EuropeanOption(payoff, exercise)

In [24]:
barrierEngine = ql.AnalyticBarrierEngine(stochProcess)

In [25]:
europeanEngine = ql.AnalyticEuropeanEngine(stochProcess)

In [26]:
knockIn.setPricingEngine(barrierEngine)

In [27]:
knockOut.setPricingEngine(barrierEngine)
european.setPricingEngine(europeanEngine)

In [28]:
replicated = knockIn.NPV() + knockOut.NPV()

In [29]:
replicated

5.892929344701621

In [30]:
expected = european.NPV()

In [31]:
expected

5.8929293447016216

In [32]:
#barrierType,barrier,rebate,type,strike,s,q,r,t,vol25Put,volAtm,vol25Call,vol, result, tol

In [33]:
values = [
    [ql.Barrier.UpOut,1.5,0,     ql.Option.Call,1.13321,1.30265,0.0003541,0.0033871,1,0.10087,0.08925,0.08463,0.11638,0.148127, 1.0e-4]
]

In [34]:
dc = ql.Actual365Fixed()
today = ql.Date(5, ql.March, 2013)
ql.Settings.instance().evaluationDate = today

spot = ql.SimpleQuote(0.0)
qRate = ql.SimpleQuote(0.0)
qTS = ql.FlatForward(today, ql.QuoteHandle(qRate), dc)
rRate = ql.SimpleQuote(0.0)
rTS = ql.FlatForward(today, ql.QuoteHandle(rRate), dc)
vol25Put = ql.SimpleQuote(0.0)
volAtm = ql.SimpleQuote(0.0)
vol25Call = ql.SimpleQuote(0.0)

In [35]:
import math as m

In [36]:
for i in range(0, len(values)):
    spot.setValue(values[i][5])
    qRate.setValue(values[i][6])
    rRate.setValue(values[i][7])
    vol25Put.setValue(values[i][8])
    volAtm.setValue(values[i][10])
    vol25Call.setValue(values[i][11])  
    payoff = ql.PlainVanillaPayoff(values[i][3], values[i][4])
    exDate = today + int(values[i][8]*365+0.5)
    exercise = ql.EuropeanExercise(exDate)
    volAtmQuote = ql.DeltaVolQuoteHandle(ql.DeltaVolQuote(ql.QuoteHandle(volAtm),
                                                         ql.DeltaVolQuote.Fwd,
                                                         values[i][8],
                                                         ql.DeltaVolQuote.AtmDeltaNeutral))
    vol25PutQuote = ql.DeltaVolQuoteHandle(ql.DeltaVolQuote(-0.25,
                                                            ql.QuoteHandle(vol25Put),
                                                         ql.DeltaVolQuote.Fwd,
                                                         values[i][8]))
    vol25CallQuote = ql.DeltaVolQuoteHandle(ql.DeltaVolQuote(0.25,
                                                            ql.QuoteHandle(vol25Call),
                                                         ql.DeltaVolQuote.Fwd,
                                                         values[i][8]))    
    barrierOption = ql.BarrierOption(values[i][0],
                                    values[i][1],
                                    values[i][2],
                                    payoff,
                                    exercise)
    
    bsVanillaPrice = ql.blackFormula(values[i][3], values[i][4],
                 spot.value()*qTS.discount(values[i][8])/rTS.discount(values[i][8]),
                 values[i][12] * m.sqrt(values[i][8]), rTS.discount(values[i][8]))
    vannaVolgaEngine = ql.VannaVolgaBarrierEngine(volAtmQuote,
                                                vol25PutQuote,
                                                vol25CallQuote,
                                                ql.QuoteHandle(spot),
                                                ql.YieldTermStructureHandle(rTS),
                                                ql.YieldTermStructureHandle(qTS),
                                                True,
                                                bsVanillaPrice)
    barrierOption.setPricingEngine(vannaVolgaEngine)
    calculated = barrierOption.NPV()
    expected = values[i][13]
    error = abs(calculated-expected)
    print(calculated)
    print (expected)

0.1803473877434823
0.148127


In [37]:
volAtmQuote.value()

0.08925

In [38]:
bsVanillaPrice

0.1803473877434823

In [39]:
vol25Call.value()

0.08463